# Import Library

In [ ]:
import Augmentor, cv2, glob, random, numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Make Dataset

In [ ]:
capt = cv2.VideoCapture(0)

if not capt.isOpened():
    exit(0)

frameFrequency = 25
video_images = 'C:\\Users\\peija\\Downloads\\Dataset\\Rizal\\'

total_frame = 0
id = 0

while True:
    ret, frame = capt.read()
    if ret is False:
        break
    total_frame += 1
    if total_frame%frameFrequency == 0:
        id += 1
        image_name = video_images + str(id) +'.jpg'
        cv2.imwrite(image_name, frame)
        print(image_name)

cap.release()

# Augmentation Dataset

In [ ]:
def perbanyak(ini, sebanyak_ini):
    p = Augmentor.Pipeline(source_directory=ini, output_directory='.')
    p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=1)
    p.rotate(probability=0.7, max_left_rotation=12, max_right_rotation=12)
    p.zoom_random(probability=0.7, percentage_area=0.9)
    p.resize(probability=1, width=64, height=64)
    p.sample(sebanyak_ini)

perbanyak("Dataset/Brandon", 350)
perbanyak("Dataset/Fayad", 350)
perbanyak("Dataset/Luqman", 350)
perbanyak("Dataset/Riski", 350)
perbanyak("Dataset/Rizal", 350)
perbanyak("Dataset/Yuga", 350)

# Model Face Classification

In [ ]:
x = []
y = []

for alamat_gambar in glob.glob("Dataset/Brandon/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(0)

for alamat_gambar in glob.glob("Dataset/Fayad/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(1)

for alamat_gambar in glob.glob("Dataset/Luqman/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(2)

for alamat_gambar in glob.glob("Dataset/Riski/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(3)

for alamat_gambar in glob.glob("Dataset/Rizal/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(4)

for alamat_gambar in glob.glob("Dataset/Yuga/*.jpg"):
    gambar = cv2.imread(alamat_gambar)
    gambar = cv2.resize(gambar, (64,64))/255
    x.append(gambar)
    y.append(5)

z = list(zip(x,y))

random.shuffle(z)

x, y = zip(*z)

x = np.asarray(x)
y = np.asarray(y)

In [ ]:
model = Sequential()

model.add(Conv2D(4, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())

model.add(Dense(10, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 4)         112       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 4)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 8)         296       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 16)        1168      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 16)         0

In [ ]:
model.fit(x, y, batch_size=1, epochs=5, validation_split=0.1)

Epoch 1/5
2028/2028 [==============================] - 8s 4ms/step - loss: 0.3366 - accuracy: 0.8698 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch 2/5
2028/2028 [==============================] - 8s 4ms/step - loss: 0.0263 - accuracy: 0.9956 - val_loss: 8.0233e-04 - val_accuracy: 1.0000
Epoch 3/5
2028/2028 [==============================] - 7s 4ms/step - loss: 0.0226 - accuracy: 0.9956 - val_loss: 7.4195e-04 - val_accuracy: 1.0000
Epoch 4/5
2028/2028 [==============================] - 7s 4ms/step - loss: 0.0172 - accuracy: 0.9961 - val_loss: 5.0043e-04 - val_accuracy: 1.0000
Epoch 5/5
2028/2028 [==============================] - 8s 4ms/step - loss: 2.1562e-04 - accuracy: 1.0000 - val_loss: 1.7074e-04 - val_accuracy: 1.0000


In [ ]:
kamus = {0:'Brandon', 1:'Fayad', 2:'Luqman', 3:'Riski', 4:'Rizal', 5:'Yuga'}
x_test = []

ini = cv2.resize(cv2.imread("Dataset/test.jpg"), (64,64))/255

x_test.append(ini)
x_test = np.asarray(x_test)

y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

[kamus[_] for _ in y_pred]

1/1 [==============================] - 0s 164ms/step


['Yuga']

# Detecting Whose Face

In [ ]:
capt = cv2.VideoCapture(0)
detek = cv2.CascadeClassifier('haarcascade_frontalface.xml')

while True:
    ret, v = capt.read()
    v = cv2.flip(v, 1)
    wajah = detek.detectMultiScale(v)
    for muka in wajah:
        x, y, w, h = muka
        wajah = v[y:y+h, x:x+w]
        wajah = cv2.resize(wajah, (64, 64))
        wajah = wajah.reshape(1, 64, 64, 3)
        y_pred = model.predict(wajah)
        kamus = {0:'Brandon',
                 1:'Fayad',
                 2:'Luqman',
                 3:'Riski',
                 4:'Rizal',
                 5:'Yuga'}
        y_pred = np.argmax(y_pred, axis=1)
        y_pred = [kamus[i] for i in y_pred]
        v = cv2.putText(v, str(y_pred), (x+w//2, y), 7, 1, (0,0,225), 1)
        v = cv2.rectangle(v, (x, y), (x+w, y+h), (255,255,255), 5)
    cv2.imshow('Whos Detected', v)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
capt.release()

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step
